In [120]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [121]:
# get data files
# !wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

# !unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [122]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [123]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [124]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [125]:
# merge dataframes on isbn
df = pd.merge(df_books, df_ratings, on='isbn', how='left')

print(df.shape)
df.head()

(1032384, 5)


,isbn,title,author,user,rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2.0,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,8.0,5.0
2,0002005018,Clara Callan,Richard Bruce Wright,11400.0,0.0
3,0002005018,Clara Callan,Richard Bruce Wright,11676.0,8.0
4,0002005018,Clara Callan,Richard Bruce Wright,41385.0,0.0


In [126]:
# get users who left more than 200 reviews
# get books with more than 100 reviews
users = df['user'].value_counts()
users = users[users > 200].index

books = df['isbn'].value_counts()
books = books[books > 100].index

In [127]:
# make new df
df = df.loc[(df['user'].isin(users.values)) & (df['isbn'].isin(books.values))]

In [128]:
# drop duplicates
df = df.drop_duplicates(['user', 'title'])

In [129]:
df.isna().sum()

isbn      0
title     0
author    0
user      0
rating    0
dtype: int64

In [130]:
df = df.dropna(how='any')

In [131]:
df_pivot = df.pivot(index='title', columns='user', values='rating').fillna(0)

In [132]:
df_matrix = csr_matrix(df_pivot.values)

In [133]:
# model creation and training
nn = NearestNeighbors(metric='cosine')
nn.fit(df_matrix)

NearestNeighbors(metric='cosine')

In [134]:
def get_recommends(book):
    # Create a list to put our model output in
    recommended_books = [book, []]

    # Pass through our book
    distance, book_info = nn.kneighbors([df_pivot.loc[book]], 5, return_distance=True)

    # Gather the text and distance and reverse the order
    recom_book_info = df_pivot.iloc[np.flip(book_info[0])[:-1]].index.to_list()
    recom_distance = list(np.flip(distance[0])[:-1])

    # For each value in our 2 variables append to our empty list above
    for r, d in zip(recom_book_info, recom_distance):
        recommended_books[1].append((r, round(d, 2)))

    return recommended_books

In [135]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False

  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']

  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  
  for i in range(2):
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [('Tis: A Memoir', 0.79), ('The Weight of Water', 0.76), ('I Know This Much Is True', 0.76), ('The Lovely Bones: A Novel', 0.73)]]
You passed the challenge! 🎉🎉🎉🎉🎉


After importing and cleaning the data, use NearestNeighbors from sklearn.neighbors to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures the distance to determine the “closeness” of instances.

Create a function named get_recommends that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

```bash
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
```

should return:

```bash
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from get_recommends() is a list. The first element in the list is the book title passed into the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed into the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.